In [ ]:
%pip install numpy==1.24.4
%pip install numba==0.57.1
%pip install shap==0.46.0
%pip install optuna==3.6.1
%pip install scikit-learn==1.3.2
%pip install seaborn==0.13.1
%pip install python-dotenv
%pip install polars

In [ ]:
import pandas as pd
import numpy as np
import lightgbm as lgb
import shap
import seaborn as sns
import matplotlib.pyplot as plt
import polars as pl

from sklearn.model_selection import train_test_split
from sklearn.model_selection import ShuffleSplit, StratifiedShuffleSplit
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer

import lightgbm as lgb

import optuna
from optuna.visualization import plot_optimization_history, plot_param_importances, plot_slice, plot_contour

from time import time

from dotenv import load_dotenv

import pickle

import os


In [ ]:
load_dotenv()

# Accedo a variables de entorno
dataset_path = os.getenv('DATASET_PATH')
dataset_file = os.getenv('DATASET_RAW_FILE')

# Cargo el dataset
dataset = pl.read_csv(dataset_path + dataset_file, ignore_errors=True)

In [ ]:
dsimple = dataset.select(
    [
        "numero_de_cliente",
        ((pl.col("foto_mes") // 100) * 12 + pl.col("foto_mes") % 100).alias("periodo0"),
        pl.arange(0, dataset.height, step=1).alias("pos"),
    ]
)

dsimple = dsimple.sort("numero_de_cliente", "periodo0")


# calculo topes
periodo_ultimo = dsimple.select(pl.max("periodo0"))
periodo_anteultimo = periodo_ultimo - 1


dsimple = dsimple.select(
    [
        pl.all(),
        pl.col("periodo0").shift(-1).over("numero_de_cliente").alias("periodo1"),
        pl.col("periodo0").shift(-2).over("numero_de_cliente").alias("periodo2"),
    ]
)


# assign most common class values = "CONTINUA"
dsimple = dsimple.with_columns(
    pl.when(pl.col("periodo0") < periodo_anteultimo)
    .then(pl.lit("CONTINUA"))
    .alias("clase_ternaria")
)


dsimple = dsimple.with_columns(
    pl.when(
        (pl.col("periodo0") < periodo_ultimo)
        & (
            pl.col("periodo1").is_null()
            | ((pl.col("periodo0") + 1) < pl.col("periodo1"))
        )
    )
    .then(pl.lit("BAJA+1"))
    .otherwise(pl.col("clase_ternaria"))
    .alias("clase_ternaria")
)


dsimple = dsimple.with_columns(
    pl.when(
        (pl.col("periodo0") < periodo_anteultimo)
        & ((pl.col("periodo0") + 1) == pl.col("periodo1"))
        & (
            pl.col("periodo2").is_null()
            | ((pl.col("periodo0") + 2) < pl.col("periodo2"))
        )
    )
    .then(pl.lit("BAJA+2"))
    .otherwise(pl.col("clase_ternaria"))
    .alias("clase_ternaria")
)


dsimple.filter(pl.col("clase_ternaria") == "BAJA+1").count()
dsimple.filter(pl.col("clase_ternaria") == "BAJA+2").count()


# pego el resultado en el dataset original y grabo
dsimple = dsimple.sort("pos")
dataset = dataset.with_columns([dsimple["clase_ternaria"]])

# Guardo dataset
#dataset.write_csv("competencia_01_polars.csv", separator=",")

In [ ]:
# Get unique values of 'clase_ternaria'
unique_values = dataset.get_column('clase_ternaria').unique()
print("Unique values:", unique_values)

# Get value counts of 'clase_ternaria'
#category_counts = dataset.groupby("clase_ternaria").count()
#category_counts = dataset.groupby("clase_ternaria").agg(pl.count().alias("count"))
category_counts = dataset.get_column('clase_ternaria').value_counts()
print("Category counts:")
print(category_counts)

# Preview the data (first few rows)
print("Data preview:")
print(dataset.head())

In [ ]:
'''
data['mpayroll_sobre_edad'] = data['mpayroll'] / data['cliente_edad']

# Variables de sumas
data['vm_mfinanciacion_limite'] = data[['Master_mfinanciacion_limite', 'Visa_mfinanciacion_limite']].sum(axis=1, skipna=True)
data['vm_Fvencimiento'] = data[['Master_Fvencimiento', 'Visa_Fvencimiento']].min(axis=1, skipna=True)
data['vm_Finiciomora'] = data[['Master_Finiciomora', 'Visa_Finiciomora']].min(axis=1, skipna=True)
data['vm_msaldototal'] = data[['Master_msaldototal', 'Visa_msaldototal']].sum(axis=1, skipna=True)
data['vm_msaldopesos'] = data[['Master_msaldopesos', 'Visa_msaldopesos']].sum(axis=1, skipna=True)
data['vm_msaldodolares'] = data[['Master_msaldodolares', 'Visa_msaldodolares']].sum(axis=1, skipna=True)
data['vm_mconsumospesos'] = data[['Master_mconsumospesos', 'Visa_mconsumospesos']].sum(axis=1, skipna=True)
data['vm_mconsumosdolares'] = data[['Master_mconsumosdolares', 'Visa_mconsumosdolares']].sum(axis=1, skipna=True)
data['vm_mlimitecompra'] = data[['Master_mlimitecompra', 'Visa_mlimitecompra']].sum(axis=1, skipna=True)
data['vm_madelantopesos'] = data[['Master_madelantopesos', 'Visa_madelantopesos']].sum(axis=1, skipna=True)
data['vm_madelantodolares'] = data[['Master_madelantodolares', 'Visa_madelantodolares']].sum(axis=1, skipna=True)
data['vm_fultimo_cierre'] = data[['Master_fultimo_cierre', 'Visa_fultimo_cierre']].max(axis=1, skipna=True)
data['vm_mpagado'] = data[['Master_mpagado', 'Visa_mpagado']].sum(axis=1, skipna=True)
data['vm_mpagospesos'] = data[['Master_mpagospesos', 'Visa_mpagospesos']].sum(axis=1, skipna=True)
data['vm_mpagosdolares'] = data[['Master_mpagosdolares', 'Visa_mpagosdolares']].sum(axis=1, skipna=True)
data['vm_fechaalta'] = data[['Master_fechaalta', 'Visa_fechaalta']].max(axis=1, skipna=True)
data['vm_mconsumototal'] = data[['Master_mconsumototal', 'Visa_mconsumototal']].sum(axis=1, skipna=True)
data['vm_cconsumos'] = data[['Master_cconsumos', 'Visa_cconsumos']].sum(axis=1, skipna=True)
data['vm_cadelantosefectivo'] = data[['Master_cadelantosefectivo', 'Visa_cadelantosefectivo']].sum(axis=1, skipna=True)
data['vm_mpagominimo'] = data[['Master_mpagominimo', 'Visa_mpagominimo']].sum(axis=1, skipna=True)

# Variables de ratios
data['vmr_Master_mlimitecompra'] = data['Master_mlimitecompra'] / data['vm_mlimitecompra']
data['vmr_Visa_mlimitecompra'] = data['Visa_mlimitecompra'] / data['vm_mlimitecompra']
data['vmr_msaldototal'] = data['vm_msaldototal'] / data['vm_mlimitecompra']
data['vmr_msaldopesos'] = data['vm_msaldopesos'] / data['vm_mlimitecompra']
data['vmr_msaldopesos2'] = data['vm_msaldopesos'] / data['vm_msaldototal']
data['vmr_msaldodolares'] = data['vm_msaldodolares'] / data['vm_mlimitecompra']
data['vmr_msaldodolares2'] = data['vm_msaldodolares'] / data['vm_msaldototal']
data['vmr_mconsumospesos'] = data['vm_mconsumospesos'] / data['vm_mlimitecompra']
data['vmr_mconsumosdolares'] = data['vm_mconsumosdolares'] / data['vm_mlimitecompra']
data['vmr_madelantopesos'] = data['vm_madelantopesos'] / data['vm_mlimitecompra']
data['vmr_madelantodolares'] = data['vm_madelantodolares'] / data['vm_mlimitecompra']
data['vmr_mpagado'] = data['vm_mpagado'] / data['vm_mlimitecompra']
data['vmr_mpagospesos'] = data['vm_mpagospesos'] / data['vm_mlimitecompra']
data['vmr_mpagosdolares'] = data['vm_mpagosdolares'] / data['vm_mlimitecompra']
data['vmr_mconsumototal'] = data['vm_mconsumototal'] / data['vm_mlimitecompra']
data['vmr_mpagominimo'] = data['vm_mpagominimo'] / data['vm_mlimitecompra']



# Filtramos solo las columnas numéricas
numeric_cols = data.select_dtypes(include=[np.number])

# Reemplazo valores infinitos con NaN solo en las columnas numéricas
infinitos_qty = np.isinf(numeric_cols).sum().sum()
if infinitos_qty > 0:
    print(f"ATENCIÓN: Hay {infinitos_qty} valores infinitos en tu dataset. Serán pasados a NaN.")
    data[numeric_cols.columns] = numeric_cols.replace([np.inf, -np.inf], np.nan)

# Valvula de seguridad para evitar valores NaN
nans_qty = data.isna().sum().sum()
if nans_qty > 0:
    print(f"ATENCIÓN: Hay {nans_qty} valores NaN en tu dataset. Serán pasados arbitrariamente a 0.")
    # Reemplazo valores NaN con 0
    data.fillna(0, inplace=True)

data.head()
'''

In [ ]:
'''
# Definimos las columnas que NO vamos a utilizar para crear los lags
campitos = ["numero_de_cliente", "foto_mes", "clase_ternaria"]

# Seleccionamos todas las columnas lagueables (que no están en campitos)
cols_lagueables = [col for col in data.columns if col not in campitos]  

# Ordenamos el DataFrame por 'numero_de_cliente' y 'foto_mes'
data.sort_values(by=["numero_de_cliente", "foto_mes"], inplace=True)

# Creamos los lags de orden 1 para las columnas en cols_lagueables
#for col in cols_lagueables:
    # Creamos la columna lag1 (desplazamiento hacia abajo de 1 periodo)
#    data[f"{col}_lag1"] = data.groupby("numero_de_cliente")[col].shift(1)

# Creamos los delta lags de orden 1
#for col in cols_lagueables:
    # Calculamos la diferencia entre el valor actual y el valor lag1
#    data[f"{col}_delta1"] = data[col] - data[f"{col}_lag1"]

# Creamos un array vacío para los nuevos datos (lags y deltas)
lags = {}
deltas = {}

# Creamos los lags y deltas usando NumPy para evitar la fragmentación
for col in cols_lagueables:
    # Obtenemos el grupo por 'numero_de_cliente'
    grouped = data.groupby("numero_de_cliente")[col]
    
    # Calculamos el lag usando shift con NumPy
    lags[f"{col}_lag1"] = grouped.shift(1).to_numpy()
    
    # Calculamos el delta como la diferencia actual - lag1
    deltas[f"{col}_delta1"] = data[col].to_numpy() - lags[f"{col}_lag1"]

# Asignamos los resultados de lag y delta directamente al DataFrame
for col_lag, values_lag in lags.items():
    data[col_lag] = values_lag
    
for col_delta, values_delta in deltas.items():
    data[col_delta] = values_delta


data.head()
'''

In [ ]:
# Filter the dataset
Xtrain = dataset.filter(pl.col('foto_mes').is_in([202104, 202105, 202106]))

# Create target variable - in polars we use map_elements for lambda functions
ytrain = (Xtrain
         .get_column('clase_ternaria')
         .map_elements(lambda x: 0 if x == "CONTINUA" else 1, return_dtype=pl.Int32)
         .to_numpy())  # Convert to numpy for LightGBM

# Drop the target column
Xtrain = Xtrain.drop('clase_ternaria')

# Convert to numpy for LightGBM, preserving column names
Xtrain_numpy = Xtrain.to_numpy()

# Create LightGBM dataset
lgb_train = lgb.Dataset(Xtrain_numpy, ytrain, feature_name=Xtrain.columns)

# Parameters remain the same
params = {
    'objective': 'binary',
    'learning_rate': 0.01,
    'verbose': 2,
    'max_bin': 15,
    'min_data_in_leaf': 500,
    'verbose': 0,
}

# Train model
gbm = lgb.train(params,
                lgb_train,
                num_boost_round=100)

# Create importance DataFrame - converting back to pandas for compatibility
lgbm_importancia = pl.DataFrame({
    'Features': gbm.feature_name(),
    'Importances': gbm.feature_importance()
}).sort('Importances', descending=True)

# If you need it as pandas DataFrame:
# lgbm_importancia = lgbm_importancia.to_pandas()

lgbm_importancia

In [ ]:
# Calculo la importancia de las variables usando SHAP

explainer = shap.TreeExplainer(gbm)
shap_values = explainer.shap_values(Xtrain_numpy)

shap_df = pl.DataFrame(shap_values, schema=Xtrain.columns)

shap_importancia = pl.DataFrame({
    'Feature': Xtrain.columns,
    'SHAP Importance': np.abs(shap_values).mean(0)
}).sort('SHAP Importance', descending=True)

shap_importancia

In [ ]:
# Visualizo

shap.summary_plot(shap_values, Xtrain_numpy, feature_names=Xtrain.columns)


In [ ]:
dataset = dataset.drop(['cprestamos_personales', 'mprestamos_personales'])

In [ ]:
#Imprime primeras 5 filas del df

dataset.head()

In [ ]:
# Especificar la ruta completa del archivo donde deseas guardar el DataFrame
output_file = dataset_path + "competencia_02_polar.csv"

# Guardar el DataFrame como un archivo CSV en la ruta especificada
dataset.write_csv(output_file)